In [ ]:
from openai import OpenAI

import pandas as pd
import os
from tqdm import tqdm
import time

In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [4]:
# 전단지 CSV 불러오기
df = pd.read_csv("data/emart_flyer_fruits.csv")
df = df.dropna(subset=["상품명"])  # 상품명이 없는 경우 제거
df.head()


,카테고리,상품명,판매가,정상가,할인율
0,전체,파프리카 3입 (봉),"2,670","4,980",할인율\r\n46\r\n%
1,전체,햇양파 (1.8kg/망),"2,864","3,580",할인율\r\n20\r\n%
2,전체,반전참외 (15입내/봉),"8,800","11,000",할인율\r\n20\r\n%
3,전체,대추방울토마토 (1kg/팩)/하우스감귤 (800g/팩),"10,980/16,900\r\n8,784/13,520",NaN,NaN
4,전체,흙당근 (1kg/봉)/햇 세척인삼 (750g/팩),"3,180/37,480\r\n2,544/29,984",NaN,NaN


In [5]:
def is_fruit_gpt(product_name):
    prompt = f"'{product_name}'라는 상품은 과일인가요? 과일이면 '과일', 아니면 '과일 아님'만 출력해주세요."

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"GPT 요청 오류: {e}")
        return "오류"

In [6]:
fruit_flags = []
for name in tqdm(df["상품명"]):
    result = is_fruit_gpt(name)
    fruit_flags.append(result)
    time.sleep(0.7)  # OpenAI API 호출 속도 제한을 피하기 위한 대기 (필수)

df["GPT판별"] = fruit_flags
df_fruits = df[df["GPT판별"] == "과일"]

100%|██████████| 56/56 [01:14<00:00,  1.33s/it]


In [7]:
df_fruits.to_csv("11emart_flyer_fruits_filtered.csv", index=False, encoding="utf-8-sig")
print("✅ 과일 상품만 필터링 완료!")
df_fruits.head()


✅ 과일 상품만 필터링 완료!


,카테고리,상품명,판매가,정상가,할인율,GPT판별
2,전체,반전참외 (15입내/봉),"8,800","11,000",할인율\r\n20\r\n%,과일
3,전체,대추방울토마토 (1kg/팩)/하우스감귤 (800g/팩),"10,980/16,900\r\n8,784/13,520",NaN,NaN,과일
5,전체,제스프리 골드키위 점보 (10개),"12,800",NaN,NaN,과일
6,전체,친환경 생 블루베리 (200g/팩),"6,980","9,980",할인율\r\n30\r\n%,과일
7,전체,블랙 망고수박/까망 애플수박,NaN,NaN,NaN,과일
